# Build

In [1]:
import tensorflow as tf
import numpy as np
import datetime
import os
from tensorflow import keras
from tensorflow.keras import layers

In [2]:
mnist = tf.keras.datasets.mnist

(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

In [4]:
x_train=x_train.reshape(60000,28,28,1)

In [5]:
x_test=x_test.reshape(10000,28,28,1)

In [ ]:
# 一般的卷积神经网络
def create_model():
  return tf.keras.models.Sequential([
    layers.Conv2D(32,(3,3),activation='relu',input_shape=(28,28,1)),
    layers.MaxPooling2D((2,2)),
    layers.Dropout(0.5),
    layers.Conv2D(64,(3,3),activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(64,(3,3),activation='relu'),
    layers.Flatten(),
    layers.Dropout(0.2),
    layers.Dense(64),
    layers.Dense(10,activation='relu')
  ])

In [ ]:
# 全连接卷积神经网络 DenseNet
def dense_block(inputs, k):
  layer1 = layers.Conv2D(k,(3,3),activation='relu',padding='same')(inputs);
  concatenate_i1 = layers.concatenate([inputs,layer1]);
  layer2 = layers.Conv2D(k,(3,3),activation='relu',padding='same')(concatenate_i1);
  concatenate_i12 = layers.concatenate([inputs,layer1,layer2]);
  layer3 = layers.Conv2D(k,(3,3),activation='relu',padding='same')(concatenate_i12);
  concatenate_i123 = layers.concatenate([inputs,layer1,layer2,layer3]);
  layer4 = layers.Conv2D(k,(3,3),activation='relu',padding='same')(concatenate_i123);
  return layer4;

In [ ]:
def dense_net():
  k=12;

  inputs = keras.Input(shape=(28,28,1));
  block1 = dense_block(inputs, k)
  conv_layer1 = layers.Conv2D(12,(3,3),activation='relu')(block1);
  pooling_layer1 = layers.MaxPooling2D((2,2))(conv_layer1);
  block2 = dense_block(pooling_layer1, k);
  conv_layer2 = layers.Conv2D(12,(3,3),activation='relu')(block2);
  pooling_layer2 = layers.MaxPooling2D((2,2))(conv_layer2)
  flatten_layer = layers.Flatten()(pooling_layer2);
  drop_out_layer = layers.Dropout(0.5)(flatten_layer)
  dense_layer1 = layers.Dense(50, activation='relu')(drop_out_layer);
  dense_layer2 = layers.Dense(10, activation='relu')(dense_layer1);
  model=keras.Model(inputs=inputs, outputs=dense_layer2, name='DenseNet');
  return model;

In [ ]:
model=dense_net()

In [ ]:
model.summary()
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
result = model.fit(x_train, y_train, epochs=30,validation_data=(x_test,y_test))